
# Amazon Reviews Data exploration


In this exercises notebook, you will be working with amazon reviews dataset hosted on AWS S3. The dataset is already downloaded during the deployment of EMR cluster. Read the data into an Sql dataframe and explore the data. 


Create an SQLContext instance. You need before you can do anything in this notebook. To create a basic instance, all we need is a SparkContext reference. Since we are running Spark in shell mode (using pySpark) we can use the global context object sc for this purpose.

In [2]:
sqlContext = SQLContext(sc)

# Dataframe in PySpark: Overview

In Apache Spark, a DataFrame is a distributed collection of rows under named columns. In simple terms, it is same as a table in relational database or an Excel sheet with Column headers. It also shares some common characteristics with RDD:

**Immutable in nature** : We can create DataFrame / RDD once but can’t change it. And we can transform a DataFrame / RDD  after applying transformations.


**Lazy Evaluations**: Which means that a task is not executed until an action is performed.


**Distributed**: RDD and DataFrame both are distributed in nature.

# Loading From json

In [ ]:
df = sqlContext.read.json("/user/hadoop/Datasets/reviews.json")

Now let's view the schema of our data.

In [ ]:
df.printSchema()

For getting the columns name we can use columns on DataFrame, similar to what we do for getting the columns in pandas DataFrame. Let’s first print the number of columns and columns name  in train file then in test file.

In [ ]:
print(len(df.columns))
df.columns

# Exploring the Data

Our data is now loaded into a dataframe that we named df, with all the dtypes inferred. First we'll count the number of rows it found:

In [ ]:
df.count()

Then we look at the column-by-column dtypes the system estimated:

In [ ]:
df.dtypes

For each pairing (a tuple object in Python, denoted by the parentheses), the first entry is the column name and the second is the dtype.

Take a peak at five rows:

In [ ]:
df.take(2)

In the format column_name=value for each row. Note that the formatting above is ugly because take doesn't try to make it pretty, it just returns the row object itself. We can use show instead and that attempts to format the data better, but because there are so many columns in this case the formatting of show doesn't fit, and each line wraps down to the next

In [ ]:
df.show(2,truncate= True)

# Describe

Now we'll describe the data. Note that describe returns a new dataframe with the information, and so must have show called after it if our goal is to view it (note the nice formatting in this case). This can be called on one or more specific columns, as we do here, or the entire dataframe by passing no columns to describe:

In [ ]:
df_described = df.describe('helpfuless_count', 'helpfuless_score', 'price')
df_described.show()

# Subsetting by Columns

One of the simplest subsettings is done by selecting just a few of the columns:

To subset the columns, we need to use select operation on DataFrame and we need to pass the columns names separated by commas inside select Operation.

In [ ]:
from pyspark.sql.functions import col

df_select = df.select(col('userId'),col('score')).show(5)
df_select.show(5)

Note that show defaults to showing the first 20 rows, but here we've specified only 5. There is also a shortcut for this notation that does the same thing but is a little easier to read. We show both because they both show up frequently in Spark resources:

In [ ]:
df_select = df[['userID', 'score']]
df_select.show(5)

Or we can do the same thing by dropping, which is convenient if we want to keep more columns than we want to drop:

In [ ]:
df_drop = df_select.drop(col('score'))

In [ ]:
df_drop.show(5)

# Subsetting by Rows

We often want to subset by rows also, for example by specifying a conditional. Note that we have to use .show() at the end of .describe(), because .describe() returns a new dataframe with the information. 

In [ ]:
df.describe('score').show()

In [ ]:
df_sub = df.where(df['score'] < 4.172)

In [ ]:
df_sub.show(5)

We can repeat the same procedure for multiple conditions and columns using standard logical operators:

In [ ]:
df_filter = df.where((df['score'] > 1) & (df['score'] < 4))

In [ ]:
df_filter.show(5)

# Random Sampling

And finally, you might want to take a random sample of rows. This can be particularlly useful, for example, if your data is large enough to require more expensive clusters to be spun up to work with it all, and you want to use a smaller, less expensive cluster to work on a sample. Once your code is completed, you can then spin up the more expensive cluster and simply apply your code to the full sample.

You can pass three arguments into sample: **the first is a boolean, which is True to sample with replacement, False without**. The second is the **fraction of the dataset** to take, in this case 5%, and the third is an **optional random seed**. If you specify any integer here then someone else performing the same random operation that specifies the same seed will get the same result. If no seed is passed then the exact random sampling can't be duplicated.

In [ ]:
df_sample = df.sample(False, 0.05, 99)

In [ ]:
df_sample.describe('score').show()

If you compare this to our original summary stats on unfiltered column userId, you'll see it does a pretty good job maintaining the mean and stddev in a sample of only 5% of the data.

# Counting the distinct rows
The distinct operation can be used here, to calculate the number of distinct rows in a DataFrame. Let’s apply distinct operation to calculate the number of distinct products in our dataset.

In [ ]:
df.select('productId').distinct().count()

# Group by a coulmn

Here is how you can group the data by a column. We used the "userId" column to group the reviews data.

In [ ]:
df.groupBy("userId").count().show()

## Running SQL Queries Programmatically

The sql function on a SQLContext enables applications to run SQL queries programmatically and returns the result as a DataFrame. We use registerTempTable() to register the "Reviews" RDD as a table. The lifetime of this temporary table is tied to the SQLContext that was used to create this DataFrame.

In [ ]:
# Registers the Reviews RDD as a table.
df.registerTempTable("Reviews")

In [ ]:
result = sqlContext.sql("SELECT count(*) as count FROM Reviews")

result.show()

In [ ]:
result = sqlContext.sql("SELECT userId, max(score) FROM Reviews group by userId")
result.show()

# You are done

Please go back to the EMR_Deploy notebook and run the remaining code cells to clode the cluster.